In [1]:
"""
Sustainable Agriculture AI for Crop Yield Prediction
======================================================

This script demonstrates how to build a machine learning model for predicting crop yields
based on environmental and agricultural factors. This can help farmers make more sustainable
decisions about resource allocation, planting schedules, and farming practices.

The model uses features like:
- Temperature (avg, min, max)
- Rainfall
- Soil moisture
- Soil pH
- Nitrogen, Phosphorus, Potassium levels
- Sustainable practices (organic fertilizer, crop rotation, etc.)

"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

# Function to generate synthetic dataset for crop yield prediction
def generate_crop_dataset(n_samples=1000):
    np.random.seed(42)
    
    # Generate features that affect crop yield
    temperature_avg = np.random.normal(25, 5, n_samples)  # Average temperature (°C)
    temperature_min = temperature_avg - np.random.normal(7, 2, n_samples)  # Min temperature
    temperature_max = temperature_avg + np.random.normal(7, 2, n_samples)  # Max temperature
    
    rainfall = np.random.gamma(shape=2, scale=25, size=n_samples)  # Rainfall (mm)
    soil_moisture = np.random.normal(35, 10, n_samples)  # Soil moisture (%)
    soil_ph = np.random.normal(6.5, 0.5, n_samples)  # Soil pH
    
    # Soil nutrients (kg/ha)
    nitrogen = np.random.normal(120, 30, n_samples)
    phosphorus = np.random.normal(15, 5, n_samples)
    potassium = np.random.normal(200, 50, n_samples)
    
    # Sustainable practices (binary)
    organic_fertilizer = np.random.binomial(1, 0.4, n_samples)
    crop_rotation = np.random.binomial(1, 0.6, n_samples)
    cover_crops = np.random.binomial(1, 0.3, n_samples)
    
    # Generate crop yield (target variable) with some noise and interactions
    base_yield = 5000 + (temperature_avg * 50) - (np.abs(temperature_avg - 22) * 80)  # Optimal temp around 22°C
    base_yield += rainfall * 10 - (0.1 * rainfall**2)  # Rainfall effect (too much is bad)
    base_yield += (soil_moisture * 20) - (soil_moisture**2 * 0.1)  # Soil moisture effect
    base_yield += -500 * ((soil_ph - 6.5)**2) + 100  # pH effect (optimal around 6.5)
    
    # Nutrient effects
    base_yield += nitrogen * 5 - (0.01 * nitrogen**2)
    base_yield += phosphorus * 15 - (0.1 * phosphorus**2)
    base_yield += potassium * 2 - (0.003 * potassium**2)
    
    # Sustainable practices effects
    base_yield += organic_fertilizer * 300
    base_yield += crop_rotation * 400
    base_yield += cover_crops * 250
    
    # Add interaction effects
    base_yield += (organic_fertilizer * nitrogen * 0.5)
    base_yield += (crop_rotation * soil_ph * 100)
    
    # Add some random noise
    noise = np.random.normal(0, 300, n_samples)
    crop_yield = base_yield + noise
    
    # Ensure yield is positive
    crop_yield = np.maximum(crop_yield, 500)
    
    # Create a DataFrame
    data = pd.DataFrame({
        'temperature_avg': temperature_avg,
        'temperature_min': temperature_min,
        'temperature_max': temperature_max,
        'rainfall': rainfall,
        'soil_moisture': soil_moisture,
        'soil_ph': soil_ph,
        'nitrogen': nitrogen,
        'phosphorus': phosphorus,
        'potassium': potassium,
        'organic_fertilizer': organic_fertilizer,
        'crop_rotation': crop_rotation,
        'cover_crops': cover_crops,
        'yield_kg_per_ha': crop_yield
    })
    
    return data

# Generate dataset
df = generate_crop_dataset(n_samples=1000)

# Save the dataset to CSV
df.to_csv('crop_yield_data.csv', index=False)
print("Dataset generated and saved to 'crop_yield_data.csv'")
print(f"Dataset shape: {df.shape}")
print("\nSample data:")
print(df.head())

# Data exploration
print("\nData summary:")
print(df.describe())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Visualize correlations
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Crop Yield Factors')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.close()

# Visualize key relationships
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

# Temperature vs Yield
sns.scatterplot(x='temperature_avg', y='yield_kg_per_ha', data=df, ax=axes[0])
axes[0].set_title('Temperature vs Yield')

# Rainfall vs Yield
sns.scatterplot(x='rainfall', y='yield_kg_per_ha', data=df, ax=axes[1])
axes[1].set_title('Rainfall vs Yield')

# Soil pH vs Yield
sns.scatterplot(x='soil_ph', y='yield_kg_per_ha', data=df, ax=axes[2])
axes[2].set_title('Soil pH vs Yield')

# Soil Moisture vs Yield
sns.scatterplot(x='soil_moisture', y='yield_kg_per_ha', data=df, ax=axes[3])
axes[3].set_title('Soil Moisture vs Yield')

# Nitrogen vs Yield
sns.scatterplot(x='nitrogen', y='yield_kg_per_ha', data=df, ax=axes[4])
axes[4].set_title('Nitrogen vs Yield')

# Sustainable practices
sns.boxplot(x='organic_fertilizer', y='yield_kg_per_ha', data=df, ax=axes[5])
axes[5].set_title('Organic Fertilizer Impact')
axes[5].set_xlabel('Organic Fertilizer (1=Yes, 0=No)')

# Crop rotation
sns.boxplot(x='crop_rotation', y='yield_kg_per_ha', data=df, ax=axes[6])
axes[6].set_title('Crop Rotation Impact')
axes[6].set_xlabel('Crop Rotation (1=Yes, 0=No)')

# Cover crops
sns.boxplot(x='cover_crops', y='yield_kg_per_ha', data=df, ax=axes[7])
axes[7].set_title('Cover Crops Impact')
axes[7].set_xlabel('Cover Crops (1=Yes, 0=No)')

# Distribution of yield
sns.histplot(df['yield_kg_per_ha'], kde=True, ax=axes[8])
axes[8].set_title('Distribution of Crop Yield')

plt.tight_layout()
plt.savefig('feature_relationships.png')
plt.close()

# Prepare data for modeling
X = df.drop('yield_kg_per_ha', axis=1)
y = df['yield_kg_per_ha']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nTraining Random Forest model...")

# Train a Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                          cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Get the best model
best_rf_model = grid_search.best_estimator_
print(f"Best hyperparameters: {grid_search.best_params_}")

# Make predictions
y_pred_train = best_rf_model.predict(X_train_scaled)
y_pred_test = best_rf_model.predict(X_test_scaled)

# Evaluate model performance
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print("\nModel Evaluation:")
print(f"Training RMSE: {train_rmse:.2f} kg/ha")
print(f"Testing RMSE: {test_rmse:.2f} kg/ha")
print(f"Training R²: {train_r2:.4f}")
print(f"Testing R²: {test_r2:.4f}")

# Feature importance
feature_importance = best_rf_model.feature_importances_
feature_names = X.columns

# Sort feature importances in descending order
sorted_idx = np.argsort(feature_importance)[::-1]

plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx])
plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance for Crop Yield Prediction')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

print("\nTop 5 most important features:")
for i in range(5):
    print(f"{feature_names[sorted_idx[i]]}: {feature_importance[sorted_idx[i]]:.4f}")

# Permutation importance (alternative importance metric)
perm_importance = permutation_importance(best_rf_model, X_test_scaled, y_test, n_repeats=10, random_state=42)
perm_importance_mean = perm_importance.importances_mean
perm_sorted_idx = np.argsort(perm_importance_mean)[::-1]

plt.figure(figsize=(10, 6))
plt.barh(range(len(perm_sorted_idx)), perm_importance_mean[perm_sorted_idx])
plt.yticks(range(len(perm_sorted_idx)), [feature_names[i] for i in perm_sorted_idx])
plt.xlabel('Permutation Importance')
plt.title('Permutation Feature Importance for Crop Yield Prediction')
plt.tight_layout()
plt.savefig('permutation_importance.png')
plt.close()

# Function to make predictions for new data
def predict_yield(model, scaler, new_data):
    """
    Make crop yield predictions for new data.
    
    Parameters:
    - model: Trained model
    - scaler: Fitted scaler
    - new_data: DataFrame with the same features as training data
    
    Returns:
    - Predicted yield in kg/ha
    """
    # Scale the new data
    new_data_scaled = scaler.transform(new_data)
    
    # Make predictions
    predictions = model.predict(new_data_scaled)
    
    return predictions

# Example usage of the predict function
print("\nExample Predictions for Different Scenarios:")

# Create example scenarios
scenarios = pd.DataFrame([
    # Baseline scenario
    {
        'temperature_avg': 24, 'temperature_min': 18, 'temperature_max': 30,
        'rainfall': 50, 'soil_moisture': 35, 'soil_ph': 6.5,
        'nitrogen': 120, 'phosphorus': 15, 'potassium': 200,
        'organic_fertilizer': 0, 'crop_rotation': 0, 'cover_crops': 0
    },
    # Sustainable farming scenario
    {
        'temperature_avg': 24, 'temperature_min': 18, 'temperature_max': 30,
        'rainfall': 50, 'soil_moisture': 35, 'soil_ph': 6.5,
        'nitrogen': 100, 'phosphorus': 15, 'potassium': 200,
        'organic_fertilizer': 1, 'crop_rotation': 1, 'cover_crops': 1
    },
    # Drought scenario
    {
        'temperature_avg': 28, 'temperature_min': 20, 'temperature_max': 36,
        'rainfall': 20, 'soil_moisture': 20, 'soil_ph': 6.5,
        'nitrogen': 120, 'phosphorus': 15, 'potassium': 200,
        'organic_fertilizer': 0, 'crop_rotation': 0, 'cover_crops': 0
    },
    # Drought with sustainable practices
    {
        'temperature_avg': 28, 'temperature_min': 20, 'temperature_max': 36,
        'rainfall': 20, 'soil_moisture': 20, 'soil_ph': 6.5,
        'nitrogen': 100, 'phosphorus': 15, 'potassium': 200,
        'organic_fertilizer': 1, 'crop_rotation': 1, 'cover_crops': 1
    }
])

# Name the scenarios for clarity
scenario_names = [
    "Baseline", 
    "Sustainable Farming", 
    "Drought Conditions", 
    "Drought with Sustainable Practices"
]

# Make predictions
predictions = predict_yield(best_rf_model, scaler, scenarios)

# Display the results
for i, (name, prediction) in enumerate(zip(scenario_names, predictions)):
    print(f"{name}: {prediction:.2f} kg/ha")
    
    # If it's the sustainable scenario, calculate the improvement
    if i == 1:
        improvement = ((prediction - predictions[0]) / predictions[0]) * 100
        print(f"  Improvement over baseline: {improvement:.2f}%")
    # If it's the drought with sustainable practices
    elif i == 3:
        improvement = ((prediction - predictions[2]) / predictions[2]) * 100
        print(f"  Improvement over drought without sustainable practices: {improvement:.2f}%")

# Save the model
import joblib
joblib.dump(best_rf_model, 'crop_yield_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("\nModel and scaler saved to 'crop_yield_model.pkl' and 'scaler.pkl'")

print("\nSustainable Agriculture AI Model Training Complete!")

# Function to provide sustainable farming recommendations based on predictions
def get_sustainability_recommendations(input_data, predicted_yield):
    """
    Generate sustainable farming recommendations based on input data and yield predictions.
    
    Parameters:
    - input_data: DataFrame row containing environmental and farming practice data
    - predicted_yield: Predicted yield for this data
    
    Returns:
    - Dictionary of recommendations
    """
    recommendations = []
    
    # Check temperature conditions
    if input_data['temperature_avg'] > 28:
        recommendations.append("Consider heat-resistant crop varieties and increase irrigation due to high temperatures.")
    
    # Check rainfall and moisture
    if input_data['rainfall'] < 30:
        recommendations.append("Implement water conservation practices: mulching, drip irrigation, and rainwater harvesting.")
        if input_data['cover_crops'] == 0:
            recommendations.append("Add cover crops to improve soil moisture retention during dry periods.")
    
    # Check soil pH
    if input_data['soil_ph'] < 6.0:
        recommendations.append("Soil is acidic. Consider applying agricultural lime to raise pH to the optimal 6.2-6.8 range.")
    elif input_data['soil_ph'] > 7.0:
        recommendations.append("Soil is alkaline. Consider adding organic matter or sulfur-containing amendments to lower pH.")
    
    # Check nutrients
    if input_data['nitrogen'] > 150:
        recommendations.append("Nitrogen levels are high. Reduce synthetic fertilizer application and consider leguminous cover crops.")
    elif input_data['nitrogen'] < 80:
        recommendations.append("Nitrogen levels are low. Consider leguminous cover crops or organic nitrogen sources.")
    
    # Check sustainable practices
    if input_data['organic_fertilizer'] == 0:
        recommendations.append("Switch to organic fertilizers to improve soil health and reduce environmental impact.")
    if input_data['crop_rotation'] == 0:
        recommendations.append("Implement crop rotation to break pest cycles and improve soil nutrients.")
    if input_data['cover_crops'] == 0:
        recommendations.append("Plant cover crops during off-seasons to prevent erosion and build soil organic matter.")
    
    return recommendations

# Generate recommendations for the example scenarios
print("\nSustainability Recommendations:")
for i, scenario_name in enumerate(scenario_names):
    print(f"\n{scenario_name} Scenario:")
    recommendations = get_sustainability_recommendations(scenarios.iloc[i], predictions[i])
    for j, rec in enumerate(recommendations, 1):
        print(f"{j}. {rec}")

Dataset generated and saved to 'crop_yield_data.csv'
Dataset shape: (1000, 13)

Sample data:
   temperature_avg  temperature_min  temperature_max    rainfall  \
0        27.483571        17.684860        33.133214    5.443222   
1        24.308678        15.459411        31.019641   19.609908   
2        28.238443        21.119182        33.653603   29.692438   
3        32.615149        26.909023        38.999226  137.110618   
4        23.829233        15.432786        27.042004   59.404363   

   soil_moisture   soil_ph    nitrogen  phosphorus   potassium  \
0      34.859010  6.836260   67.162723   16.597771  247.447411   
1      41.802155  6.109496  127.774010   12.386125  262.140054   
2      33.011503  6.629942  105.459982   17.513986  280.885854   
3      17.921621  6.027068  101.401329   20.242424  179.925416   
4      38.324681  6.629979  102.431345   10.070948  176.277475   

   organic_fertilizer  crop_rotation  cover_crops  yield_kg_per_ha  
0                   0           